# WORD EMBEDDINGS: ENCODING LEXICAL SEMANTICS

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

## 1. Data

In [2]:
# Context size.
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10

In [3]:
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [4]:
# Creates the trigram
trigrams = [([test_sentence[i], 
            test_sentence[i + 1]], 
            test_sentence[i + 2]) for i in range(len(test_sentence) - 2)]
for trigram in trigrams:
    print(trigram)

(['When', 'forty'], 'winters')
(['forty', 'winters'], 'shall')
(['winters', 'shall'], 'besiege')
(['shall', 'besiege'], 'thy')
(['besiege', 'thy'], 'brow,')
(['thy', 'brow,'], 'And')
(['brow,', 'And'], 'dig')
(['And', 'dig'], 'deep')
(['dig', 'deep'], 'trenches')
(['deep', 'trenches'], 'in')
(['trenches', 'in'], 'thy')
(['in', 'thy'], "beauty's")
(['thy', "beauty's"], 'field,')
(["beauty's", 'field,'], 'Thy')
(['field,', 'Thy'], "youth's")
(['Thy', "youth's"], 'proud')
(["youth's", 'proud'], 'livery')
(['proud', 'livery'], 'so')
(['livery', 'so'], 'gazed')
(['so', 'gazed'], 'on')
(['gazed', 'on'], 'now,')
(['on', 'now,'], 'Will')
(['now,', 'Will'], 'be')
(['Will', 'be'], 'a')
(['be', 'a'], "totter'd")
(['a', "totter'd"], 'weed')
(["totter'd", 'weed'], 'of')
(['weed', 'of'], 'small')
(['of', 'small'], 'worth')
(['small', 'worth'], 'held:')
(['worth', 'held:'], 'Then')
(['held:', 'Then'], 'being')
(['Then', 'being'], 'asked,')
(['being', 'asked,'], 'where')
(['asked,', 'where'], 'all')
(

In [5]:
vocab = set(test_sentence)
word_to_index = {word : i for i, word in enumerate(vocab)}

## 2. Model

In [6]:
class NGramLanguageModeler(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)
        
    def forward(self, inputs):
        embeds = self.embeddings(inputs).view(1, -1)
        
        z1 = self.linear1(embeds)
        a1 = F.relu(z1)
        
        z2 = self.linear2(a1)
        a2 = F.log_softmax(z2, dim=1)
        
        return a2

In [7]:
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)

## 3. Loss Function.

In [8]:
losses = []
loss_function = nn.NLLLoss()

## 4. Optimizer

In [9]:
optimizer = optim.SGD(model.parameters(), lr=0.001)

## 5. Train the Model

In [11]:
for epoch in range(10):
    total_loss = 0
    
    for context, target in trigrams:
        model.zero_grad()
        
        # Prepares data.
        context_indices = torch.tensor([word_to_index[w] for w in context], 
                                       dtype=torch.long)
        
        # Forward prop.
        log_probs = model(context_indices)
        loss = loss_function(log_probs, torch.tensor([word_to_index[target]], 
                                                     dtype=torch.long))
        
        # Backward prop.
        loss.backward()
        
        # Updates the params.
        optimizer.step()
        
        total_loss += loss.item()
    losses.append(total_loss)

print(losses)

[518.5732445716858, 516.1532635688782, 513.7477684020996, 511.3560872077942, 508.9777936935425, 506.61112451553345, 504.2554540634155, 501.91208124160767, 499.5782382488251, 497.25394582748413, 494.9389293193817, 492.63220477104187, 490.331910610199, 488.03724455833435, 485.74853467941284, 483.46796464920044, 481.1920232772827, 478.91941952705383, 476.6516783237457, 474.38761734962463]
